In [ ]:
# apps/overview_limits/views.py
"""Provide several modules.

This module allows the user to review over_view_limits.

Examples:
    >>> VolFlowChart(df, rail_pressure, factor, internextern)
    


The module contains the following functions:

- `VolFlowChart(df, rail_pressure, factor, internextern)` - Returns the Volume flow chart of the cam.

"""

from typing import Union
# ...
from django.shortcuts import render
from django.http import HttpResponse
from apps.main.models import D00DriveRequest,RequestHeader,I2Customer, I3Prj, I4PartNumber, ComboBox, Cfh1DeliveryRateHeader, Cfd1DeliveryRateData, Pfh1DeliveryRateHeader, D3CamHeader, D5CamKinematicLawHeader, D6CamKinematicLawData, Pfd1DeliveryRateData, H0Hdp, H5_Piston, H2_DBV, ParameterType,Overviewlimits
from django.db.models import Max
from django.views import View
from django.contrib.auth.decorators import login_required, user_passes_test
from django.utils.decorators import method_decorator
from .serializers import D00DriveRequestSerializer, D1DriveSpringSerializer, D2DriveSpringDiscSerializer, D4DriveLifterSerializer, D5CamKinematicLawHeaderSerializer
import pandas as pd
from datatidying.Custompyfunc import dftohtml
import json
from .customfunction import TestChart, VolRateChart,VolFlowChart, Splitandpivot_torque, deliverytorque,exportmessdaten
import math
import numpy as np
from rest_framework.decorators import api_view, permission_classes
from rest_framework import permissions

from rest_framework.permissions import IsAuthenticated
from rest_framework import status , generics
from rest_framework import status
from rest_framework.response import Response

############# Drive Request ####################
@permission_classes((permissions.AllowAny,))
class Api_List_Collection_Drive_Request(generics.GenericAPIView):
    serializer_class = D00DriveRequestSerializer
    permission_classes = [IsAuthenticated,]

    def get(self, request):
        items = D00DriveRequest.objects.all()
        serializer = D00DriveRequestSerializer(items, many=True)
        return Response(serializer.data)
    def post(self,request):
        serializer= D00DriveRequestSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)







@permission_classes((permissions.AllowAny,))
class Api_List_Detail_Drive_Request(generics.GenericAPIView): #Drive Request Detail
    serializer_class = D00DriveRequestSerializer
    permission_classes = [IsAuthenticated,]
    def delete(self,request,pk):
        try:
            item = D00DriveRequest.objects.get(id=pk)
            item.delete()
            return JsonResponse(status=status.HTTP_204_NO_CONTENT)
        except D00DriveRequest.DoesNotExist:
            return JsonResponse(status=status.HTTP_204_NO_CONTENT)

    def put(self,request,pk):

        item = D00DriveRequest.objects.get(id=pk)
        serializer = D00DriveRequestSerializer(item, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors,status=status.HTTP_400_BAD_REQUEST)

@method_decorator(login_required, name='dispatch')
@method_decorator(user_passes_test(lambda u: u.has_perm('main.overview_limit'),login_url='permission_not_granted'), name='dispatch')
class Overviewcustomerlimit(View):
    templatename = "panel/overviewcustomerlimit.html"

    def get(self, request, *args, **kwargs):

        customer = I2Customer.objects.filter(active=1).values('i2_id', 'customer_name').distinct().order_by('customer_name')
        project = I3Prj.objects.filter(active=1).values('i3_id', 'prj_name', 'i2', 'variant_name').distinct().order_by('prj_name')
        partnumber = I4PartNumber.objects.filter(active=1).values('i4_id', 'part_number','part_number_version', 'i3', 'request__request_type').order_by('part_number','part_number_version')
        combobox = RequestHeader.objects.filter().values('r0_id', 'request_type')

        FID_dep1 = Overviewlimits.objects.filter(func_type='FID').values('dependency1_name', 'dependency1_unit').distinct
        FID_dep2 = Overviewlimits.objects.filter(func_type='FID').values('dependency2_name', 'dependency2_unit').distinct
        FID_dep3 = Overviewlimits.objects.filter(func_type='FID').values('dependency3_name', 'dependency3_unit').distinct
        FID_value = Overviewlimits.objects.filter(func_type='FID').values('value_name', 'value_unit').distinct

        DHF_dep1 = Overviewlimits.objects.filter(func_type='DHF').values('dependency1_name', 'dependency1_unit').distinct
        DHF_dep2 = Overviewlimits.objects.filter(func_type='DHF').values('dependency2_name', 'dependency2_unit').distinct
        DHF_dep3 = Overviewlimits.objects.filter(func_type='DHF').values('dependency3_name', 'dependency3_unit').distinct
        DHF_value = Overviewlimits.objects.filter(func_type='DHF').values('value_name', 'value_unit').distinct

        DBV_dep1 = Overviewlimits.objects.filter(func_type='DBV_PO').values('dependency1_name', 'dependency1_unit').distinct
        DBV_dep2 = Overviewlimits.objects.filter(func_type='DBV_PO').values('dependency2_name', 'dependency2_unit').distinct
        DBV_dep3 = Overviewlimits.objects.filter(func_type='DBV_PO').values('dependency3_name', 'dependency3_unit').distinct
        DBV_value = Overviewlimits.objects.filter(func_type='DBV_PO').values('value_name', 'value_unit').distinct

        DELIV_dep1 = Overviewlimits.objects.filter(func_type='DELIVERY_RATE').values('dependency1_name', 'dependency1_unit').distinct
        DELIV_dep2 = Overviewlimits.objects.filter(func_type='DELIVERY_RATE').values('dependency2_name', 'dependency2_unit').distinct
        DELIV_dep3 = Overviewlimits.objects.filter(func_type='DELIVERY_RATE').values('dependency3_name', 'dependency3_unit').distinct
        DELIV_value = Overviewlimits.objects.filter(func_type='DELIVERY_RATE').values('value_name', 'value_unit').distinct

        return render(request, self.templatename, {'customer': customer, 'project': project, 'partnumber': partnumber, 'combobox': combobox, 'FID_dep1': FID_dep1, 'FID_dep2': FID_dep2, 'FID_dep3': FID_dep3, 'FID_value': FID_value, 'DHF_dep1': DHF_dep1, 'DHF_dep2': DHF_dep2, 'DHF_dep3': DHF_dep3, 'DBV_dep1': DBV_dep1, 'DBV_dep2': DBV_dep2, 'DBV_dep3': DBV_dep3, 'DELIV_dep1': DELIV_dep1, 'DELIV_dep2': DELIV_dep2, 'DELIV_dep3': DELIV_dep3})

    def post(self, request, *args, **kwargs):
        if request.POST["action"] == "getoverview":
            response_data = {}
            # selectedcustomer=request.POST.get("customer")
            # selectedproject=request.POST.get("project")
            selectedpartnumber = request.POST.get("partnumber")           
            partnumbername = request.POST.get("partnumbername")
            partnumbernameversion = request.POST.get("partnumbernameversion")
            customerid = int(request.POST.get("customerid"))
            projectid = int(request.POST.get("projectid"))
            
            cust=I2Customer.objects.filter(i2_id=customerid).values('customer_name').first()
            proj=I3Prj.objects.filter(i3_id=projectid).values('prj_name', 'variant_name').first()
            
            #print(cust['customer_name'])
            #print(proj['prj_name'])
            
            modulename = ['fid', 'dhf', 'dbv', 'vol','torque', 'sup', 'partialdel', 'auvmax']
            func_type = ['FID', 'DHF', 'DBV_PO', 'DELIVERY_RATE','TORQUE', 'SUP', 'PARTIAL_DELIVERY', 'AuV_Max']

            for i in range(len(modulename)):
                if modulename[i]=='fid' or modulename[i]=='dhf' or modulename[i]=='dbv':               
                    info_query = Overviewlimits.objects.filter(func_type=func_type[i], i4id=selectedpartnumber).values('active', 'version', 'revision', 'mapid', 'comment','creation','author__last_name')
                    #print('info_query\n',info_query)
                    if info_query:
                        info_result = info_query[0]
                        response_data[modulename[i]+'info'] = {'active': info_result['active'], 'version': info_result['version'],'revision': info_result['revision'], 'mapid': info_result['mapid'], 'comment': info_result['comment'], 'creation': str(info_result['creation']),'author':info_result['author__last_name']}
                        #print('info_result\n',info_result)
                    else:
                        response_data[modulename[i]+'info'] = {'active': '','version': '', 'revision': '', 'mapid': '', 'comment': '', 'creation':''}
                        #print('deneme')
                else:
                    info_query = Overviewlimits.objects.filter(func_type=func_type[i], i4id=selectedpartnumber).values('active', 'version', 'revision', 'mapid', 'comment','creation','author__last_name')
                    if info_query:
                        info_result = info_query[0]
                        response_data[modulename[i]+'info'] = {'active': info_result['active'], 'version': info_result['version'],'revision': info_result['revision'], 'mapid': info_result['mapid'], 'comment': info_result['comment'], 'creation': str(info_result['creation']),'author':info_result['author__last_name']}
                    else:
                        response_data[modulename[i]+'info'] = {'active': '','version': '', 'revision': '', 'mapid': '', 'comment': '', 'creation':'','author':''}        

            # FID-----------------------------------------------------------------------------------------------------------------------------------------
            fidresult = Overviewlimits.objects.filter(func_type='FID',active=1, i4id=selectedpartnumber).values('dependency1', 'dependency2', 'value','value_unit','cases1')
            if fidresult:
                response_data['fiddata'] = dftohtml(pd.DataFrame(list(fidresult)))
                
            # DHF-----------------------------------------------------------------------------------------------------------------------------------------
            dhfresult = Overviewlimits.objects.filter(func_type='DHF', active=1, i4id=selectedpartnumber).values('dependency1','value','cases1')
            if dhfresult:
                response_data['dhfdata'] = dftohtml(pd.DataFrame(list(dhfresult)))

            # DBV_Pö-----------------------------------------------------------------------------------------------------------------------------------------
            dbvresult = Overviewlimits.objects.filter(func_type='DBV_PO', active=1, i4id=selectedpartnumber).values('dependency1', 'dependency2', 'dependency3','value','cases1')
            if dbvresult:
                response_data['dbvdata'] = dftohtml(pd.DataFrame(list(dbvresult)))

            # Delivery-----------------------------------------------------------------------------------------------------------------------------------------
            deliveryresult = Overviewlimits.objects.filter(func_type='DELIVERY_RATE', active=1, i4id=selectedpartnumber).values('dependency1', 'dependency2','value','cases1').exclude(cases1__contains='ENG')
            if deliveryresult:
                pistond = float(request.POST.get("pistond"))/100
                lift = float(request.POST.get("lift"))/100
                lobe = float(request.POST.get("lobe"))
                internextern = request.POST.get("internextern")
                print(internextern)

                systempressure = float(request.POST.get("systempressure"))
                coefficient = np.pi*np.power(pistond, 2)*0.25*lift*lobe*60
                deliveryresult = pd.DataFrame(list(deliveryresult))
                deliveryresult['flow'] = deliveryresult["dependency2"]*(deliveryresult["value"]/100)*coefficient
                deliveryresult=deliveryresult[['dependency1','dependency2','value','flow','cases1']] 
                #print('deliveryresult\n',deliveryresult)
                response_data['voldata'] = dftohtml(deliveryresult)

                dfvolchart = pd.DataFrame()
                dfvolchart['System Pressure'] = deliveryresult['dependency1']
                dfvolchart['Cam Speed'] = deliveryresult['dependency2']
                dfvolchart['Volumetric Efficiency'] = deliveryresult['value']
                dfvolchart['Volumetric Flow'] = deliveryresult['flow']
                dfvolchart['cases1'] = deliveryresult['cases1']
                df=dfvolchart
                df=df.drop(df[df['cases1'].str.contains('possible')].index)
                dfvolchart=df
                
                
                #print('dfvolchart\n',dfvolchart)
                #response_data['volratechartENGdiv']=TestChart()
                
                if internextern=='EXTERNAL': 
                    response_data['volratechartdiv'] = VolRateChart(dfvolchart, systempressure, coefficient,internextern)
                    response_data['volflowchartdiv'] = VolFlowChart(dfvolchart, systempressure, coefficient,internextern)
                    print(VolRateChart(dfvolchart, systempressure, coefficient,internextern))
                if internextern=='INTERNAL':
                    response_data['volratechartdiv'] = VolRateChart(dfvolchart, systempressure, coefficient,internextern)
                    response_data['volflowchartdiv'] = VolFlowChart(dfvolchart, systempressure, coefficient,internextern)
                    print(VolRateChart(dfvolchart, systempressure, coefficient,internextern))
                
                
            # Torque-----------------------------------------------------------------------------------------------------------------------------------------
            torqueresult = Overviewlimits.objects.filter(func_type='TORQUE', active=1, i4id=selectedpartnumber).values('cases1', 'dependency1', 'dependency2', 'value')
            if torqueresult:
                torqueresult = pd.DataFrame(list(torqueresult))
                torqueresult = torqueresult[['dependency1','dependency2','value','cases1']]
                response_data['torquedata'] = dftohtml(torqueresult)
                
                #torqueresult = pd.DataFrame(list(torqueresult))
                dftorquechart = pd.DataFrame()
                dftorquechart['cases1'] = torqueresult['cases1']
                dftorquechart['System Pressure'] = torqueresult['dependency1']
                dftorquechart['Cam Speed'] = torqueresult['dependency2']
                dftorquechart['Torque'] = torqueresult['value']
            
                #response_data['torquechartdiv']=TestChart()
                response_data['torquechartdiv'] = deliverytorque(Splitandpivot_torque(dftorquechart))
            
            

            # Sup-----------------------------------------------------------------------------------------------------------------------------------------
            supresult = Overviewlimits.objects.filter(func_type='SUP', active=1, i4id=selectedpartnumber).values('cases1', 'value')
            if supresult:
                response_data['supdata'] = dftohtml(pd.DataFrame(list(supresult)))

            # Partial-----------------------------------------------------------------------------------------------------------------------------------------
            internextern = request.POST.get("internextern")
            systempressure = float(request.POST.get("systempressure"))
            if internextern=='INTERNAL':
                partialdelresult = Overviewlimits.objects.filter(func_type='PARTIAL_DELIVERY', active=1, i4id=selectedpartnumber).values('cases1', 'dependency1', 'dependency2', 'value')                
                if partialdelresult:
                    #print(pd.DataFrame(list(partialdelresult)))           
                    partialdelresult = pd.DataFrame(list(partialdelresult))
                    partialdelresult['systempressure'] = systempressure
                    partialdelresult = partialdelresult[['dependency1','systempressure','dependency2','value','cases1']]
                    response_data['partialdeldata'] = dftohtml(partialdelresult)
            
            # Auv-----------------------------------------------------------------------------------------------------------------------------------------
            auvmaxresult = Overviewlimits.objects.filter(func_type='AuV_Max', active=1, i4id=selectedpartnumber).values('value_unit','value','cases1')
            if auvmaxresult:
                response_data['auvmaxdata'] = dftohtml(pd.DataFrame(list(auvmaxresult)))

            return HttpResponse(json.dumps(response_data), content_type="application/json")

        if request.POST["action"] == "getinfobypartnum":
            response_data = {}
            selectedpartnumber = request.POST.get("partnumid")

            partnuminfo = I4PartNumber.objects.filter(i4_id=int(selectedpartnumber)).values('h0hdp_id__adr', 'h0hdp_id__generation', 'h0hdp_id__sample_state', 'max_engine_speed', 'ratio_cam_to_crankshaftt', 'system_pressure', 'h0hdp_id__h5_piston__piston_diameter')
            #print(pd.DataFrame(list(partnuminfo)))
            cam_type = D3CamHeader.objects.filter(drive2cam__partnum2drive__i4_id=int(selectedpartnumber)).values('cam_part_no', 'cam_lobe_count', 'cam_lobe_lift', 'd5__cam_profile', 'd5__cam_profile_prefix')

            response_data['cam_name'] = cam_type[0]['cam_part_no']
            response_data['lobe'] = float(cam_type[0]['cam_lobe_count'])
            response_data['lift'] = round(float(cam_type[0]['cam_lobe_lift']), 1)
            response_data['generation'] = partnuminfo[0]['h0hdp_id__generation']
            response_data['sample'] = partnuminfo[0]['h0hdp_id__sample_state']
            response_data['systempressure'] = partnuminfo[0]['system_pressure']
            response_data['pistond'] = partnuminfo[0]['h0hdp_id__h5_piston__piston_diameter']
            response_data['camratio'] = partnuminfo[0]['ratio_cam_to_crankshaftt']
            response_data['maxenginespeed'] = partnuminfo[0]['max_engine_speed']

            return HttpResponse(json.dumps(response_data), content_type="application/json")

        if request.POST["action"] == "exportmessdaten":
            response_data={}
            i3id = request.POST.get("i3_id")
            Limits= pd.DataFrame(list(Overviewlimits.objects.filter(i3_id=int(i3id),active=1).values('func_type','dependency1','dependency2','request','cases1','value')))
            caminfo=pd.DataFrame(list(CamInfo.objects.filter(i3_id=int(i3id),request=8).values('internal_external','cam_part_no','cam_lobe_count','cam_lobe_lift','cam_lobe_angle_bdctcd','piston_diameter','spring_type','lifter_diameter')))
            SupStation = pd.DataFrame(list(StationParameter.objects.filter(st_name='PV test').values('cases1','value')))

            exportmessdaten(caminfo,Limits,SupStation)


            return HttpResponse(json.dumps(response_data), content_type="application/json")